# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [4]:
conda install -c conda-forge gensim

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38h50d1736_0         3.1 MB  conda-forge
    gensim-3.8.3               |   py38hc84c608_2        22.7 MB  conda-forge
    smart_open-5.1.0           |     pyhd8ed1ab_1          42 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        25.8 MB

The following NEW packages will be INSTALLED:

  gensim             conda-forge/osx-64::gensim-3.8.3-py38hc84c608_2
  smart_open         conda-forge/noarch::smart_open-5.1

In [5]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [9]:
import zipfile as z

In [10]:
target = 'News_Category_Dataset_v2.zip'

In [11]:
root = z.ZipFile(target)

In [12]:
root.extractall('new')

In [13]:
root.close()

Used this website to figure out how to unzip: https://medium.com/analytics-vidhya/unzip-any-file-using-python-9eeac7529c64

In [14]:
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [15]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [16]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [17]:
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = 4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [18]:
model.train(data, total_examples = model.corpus_count, epochs = 10)

(55588501, 67376200)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [19]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [20]:
wv.most_similar('Texas')

[('Louisiana', 0.816248893737793),
 ('Pennsylvania', 0.8124288320541382),
 ('Ohio', 0.7985190153121948),
 ('Illinois', 0.7979607582092285),
 ('Oklahoma', 0.7937803268432617),
 ('Connecticut', 0.7886255979537964),
 ('Maryland', 0.7874077558517456),
 ('Massachusetts', 0.786849319934845),
 ('Florida', 0.7847256660461426),
 ('Arkansas', 0.7811683416366577)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [21]:
wv.most_similar(negative = 'Texas')

[('Admon', 0.41727912425994873),
 ('parliamentarians', 0.3984353244304657),
 ('maitre', 0.3839201033115387),
 ('8,710', 0.37364763021469116),
 ('Rank-and-file', 0.36710697412490845),
 ("'tough", 0.3543519973754883),
 ('Hunger-Free', 0.3541926145553589),
 ('Namaz', 0.35106605291366577),
 ('Headstrong', 0.3503102660179138),
 ('NRA-Style', 0.35019439458847046)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [22]:
wv['Texas']

array([-0.2369677 , -0.86312973, -0.42725083,  0.85874295, -0.31793544,
       -1.5888221 , -1.9130476 , -2.459345  ,  0.36325967,  0.4395181 ,
        0.5334726 , -1.361529  , -0.9127467 ,  0.88662845,  0.91853166,
       -0.12098543, -2.9832785 ,  0.10236472,  0.9639339 , -1.4829004 ,
        1.0999535 , -0.47186235, -1.5107446 ,  0.71976155, -0.13398315,
       -1.3256497 ,  0.18997493,  0.6516107 ,  1.4665868 , -0.72353244,
        1.3196964 ,  1.2133355 ,  0.28366965, -3.0524964 , -0.29593098,
       -0.37438723, -0.07615288,  0.5474493 ,  1.2414538 , -0.28077126,
       -1.9337732 , -0.5077366 ,  0.09752356, -1.627838  , -0.82560647,
       -2.4812353 ,  1.1070956 , -1.6053305 , -0.8818438 ,  0.86213183,
       -0.32847732, -2.2106454 ,  0.10618958,  0.5364948 ,  2.359165  ,
        0.67302215,  0.9484168 , -1.2397262 , -0.27108696,  0.4653041 ,
       -1.5428581 ,  2.4436169 ,  1.0521102 ,  0.6118897 , -0.55451024,
        1.7756401 ,  0.2961901 ,  0.0854935 ,  1.4501396 ,  3.96

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [23]:
wv.vectors

array([[-8.9142942e-01,  9.0472364e-01, -3.9666542e-01, ...,
        -1.6555730e+00,  1.8875605e+00, -2.1402292e-01],
       [-1.8121271e+00,  3.0713151e+00, -2.7186680e+00, ...,
        -7.8025967e-01,  8.7369013e-01,  3.5777903e-01],
       [-7.8548396e-01,  1.5567520e+00, -1.1000307e+00, ...,
        -9.4454788e-02,  1.1792376e+00,  2.0824983e+00],
       ...,
       [ 6.1115738e-02, -2.5792677e-02, -4.8673511e-03, ...,
        -7.2163545e-02,  2.1184778e-02, -1.1479849e-01],
       [ 4.9576506e-02,  4.6543360e-02,  4.9832132e-02, ...,
        -2.5756380e-02,  3.0129801e-02, -3.3558436e-02],
       [-1.1146535e-02, -2.7584523e-02, -2.2746674e-03, ...,
        -1.9347358e-02, -3.0097453e-02, -5.8274366e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [26]:
wv.most_similar(positive = ['king', 'woman'], negative = ['man'])

[('tchiest', 0.5446792840957642),
 ('ssy', 0.5111991763114929),
 ('sl', 0.5063759088516235),
 ('cking', 0.5039977431297302),
 ('rn', 0.5022842884063721),
 ('tch', 0.5010480880737305),
 ("'punished", 0.48680996894836426),
 ('Whiteco', 0.4865831732749939),
 ('feminist', 0.4794636368751526),
 ('ts', 0.4780205190181732)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!